In [66]:
import requests
import re
import json
from time import sleep

## Get Google Places API Key https://developers.google.com/places/web-service/get-api-key
api_key = ''

class StopAssignments(Exception): pass

def get_google_results(r):
    output = {}
    if r.status_code == 200:
        if r.json()['status'] == 'OK':
            for x in range(len(r.json()['results'])):
                results = r.json()['results'][x]
                output[results['place_id']] = {
                    'formatted_address': results['formatted_address'],
                    'name': results['name'],
                    'loc_type': results['types'],
                    'lat': results['geometry']['location']['lat'],
                    'lon': results['geometry']['location']['lng']                                      
                }
            return output

        elif r.json()['status'] == 'ZERO_RESULTS':
            return output
            print (r.json()['status'])
            raise StopAssignments
        elif r.json()['status'] == 'OVER_QUERY_LIMIT':
            return output
            print (r.json()['status'])
            raise StopAssignments
        else:
             raise StopAssignments
    else:
        raise StopAssignments

## Helper function to go to next page of google places search results
def google_places_next_page(next_page_token, api_key):
    web_call_string = 'https://maps.googleapis.com/maps/api/place/textsearch/json?pagetoken=%s&key=%s' %(next_page_token, api_key)
    sleep(3) # Time in seconds. There is a time lag between when the next page token is issued and when it is ready
    r = requests.get(web_call_string)
    return r

## Helper function to get data from Google Places
def google_places_call(zip_code, city, keyword, api_key): ## city should be passed as "New York, NY" for NYC
    
    text_string = '%s+near+%s+%s' %(re.sub(' ', '+', keyword), str(zip_code), re.sub(' ', '+', city))
    web_call_string = 'https://maps.googleapis.com/maps/api/place/textsearch/json?query=%s&key=%s' %(text_string,api_key)
    r = requests.get(web_call_string)
    output = get_google_results(r)

    while 'next_page_token' in r.json().keys():
        r = google_places_next_page(r.json()['next_page_token'], api_key)
        new_output = get_google_results(r)
        output = dict(list(output.items()) + list(new_output.items()))
    
    return output

In [67]:
## Usage - pass zip code, city and key word along with api key
test = google_places_call(11104, 'New York, NY', 'coffee shops', api_key)

In [72]:
## The results will be a JSON with the key being the Google Places unique place id
test['ChIJvUuF1NlewokRzEqqGWT2ATY']

{'formatted_address': '40-07 Queens Blvd, Sunnyside, NY 11104, United States',
 'lat': 40.7441065,
 'loc_type': ['cafe', 'food', 'store', 'point_of_interest', 'establishment'],
 'lon': -73.9237985,
 'name': "Baruir's Coffee"}